# Loop 51 Analysis: Submission Error Investigation

**Problem:** exp_050 submission failed with "Evaluation metric raised an unexpected error"

**Goal:** Understand what's wrong with the submission format and fix it.

In [ ]:
import pandas as pd
import numpy as np

# Load our submission
sub = pd.read_csv('/home/submission/submission.csv')
print('Submission shape:', sub.shape)
print('Columns:', list(sub.columns))
print()
print('First 10 rows:')
print(sub.head(10))

In [ ]:
# Check the data types
print('Data types:')
print(sub.dtypes)
print()

# Check for any issues
print('NaN values:')
print(sub.isnull().sum())
print()

print('Infinite values:')
for col in ['target_1', 'target_2', 'target_3']:
    print(f'  {col}: {np.isinf(sub[col]).sum()}')
print()

print('Negative values:')
for col in ['target_1', 'target_2', 'target_3']:
    print(f'  {col}: {(sub[col] < 0).sum()}')

In [ ]:
# Check the row counts per fold
print('Task 0 (single solvent) - rows per fold:')
task0 = sub[sub['task'] == 0]
print(task0.groupby('fold').size())
print(f'Total: {len(task0)}')
print()

print('Task 1 (full data) - rows per fold:')
task1 = sub[sub['task'] == 1]
print(task1.groupby('fold').size())
print(f'Total: {len(task1)}')

In [ ]:
# Load the actual data to compare
DATA_PATH = '/home/data'
df_single = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
df_full = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')

print('Actual data:')
print(f'  Single solvent: {len(df_single)} rows, {df_single["SOLVENT NAME"].nunique()} solvents')
print(f'  Full data: {len(df_full)} rows, {len(df_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates())} solvent pairs')
print()

# Check if our submission has the right number of rows
print('Our submission:')
print(f'  Task 0: {len(task0)} rows (expected: {len(df_single)})')
print(f'  Task 1: {len(task1)} rows (expected: {len(df_full)})')
print(f'  Total: {len(sub)} rows (expected: {len(df_single) + len(df_full)})')

In [ ]:
# Check the row indices per fold
print('Task 0 - row indices per fold:')
for fold in sorted(task0['fold'].unique()):
    fold_data = task0[task0['fold'] == fold]
    print(f'  Fold {fold}: rows {fold_data["row"].min()}-{fold_data["row"].max()} (n={len(fold_data)})')
print()

print('Task 1 - row indices per fold:')
for fold in sorted(task1['fold'].unique()):
    fold_data = task1[task1['fold'] == fold]
    print(f'  Fold {fold}: rows {fold_data["row"].min()}-{fold_data["row"].max()} (n={len(fold_data)})')

In [ ]:
# Compare with a successful submission format
# Let's check what the template produces

# The template shows:
# - id: row index (0 to N-1)
# - index: same as id (from reset_index())
# - task: 0 for single solvent, 1 for full data
# - fold: fold index
# - row: row index within the fold
# - target_1, target_2, target_3: predictions

# Our submission has the same format
print('Checking format consistency...')
print(f'id == index: {(sub["id"] == sub["index"]).all()}')
print(f'id is sequential: {(sub["id"] == range(len(sub))).all()}')
print(f'task values: {sorted(sub["task"].unique())}')
print(f'fold values for task 0: {sorted(task0["fold"].unique())}')
print(f'fold values for task 1: {sorted(task1["fold"].unique())}')

In [ ]:
# Check if the row indices are correct
# For each fold, row should start from 0
print('Checking row indices...')
for task in [0, 1]:
    task_data = sub[sub['task'] == task]
    for fold in sorted(task_data['fold'].unique()):
        fold_data = task_data[task_data['fold'] == fold]
        if fold_data['row'].min() != 0:
            print(f'  Task {task}, Fold {fold}: row starts at {fold_data["row"].min()} (should be 0)')
        if fold_data['row'].max() != len(fold_data) - 1:
            print(f'  Task {task}, Fold {fold}: row ends at {fold_data["row"].max()} (should be {len(fold_data) - 1})')
print('Done checking row indices.')

In [ ]:
# CV-LB relationship analysis
print('='*60)
print('CV-LB RELATIONSHIP ANALYSIS')
print('='*60)

# Historical submissions
submissions = [
    ('exp_000', 0.0111, 0.0982),
    ('exp_001', 0.0123, 0.1065),
    ('exp_003', 0.0105, 0.0972),
    ('exp_005', 0.0104, 0.0969),
    ('exp_006', 0.0097, 0.0946),
    ('exp_007', 0.0093, 0.0932),
    ('exp_009', 0.0092, 0.0936),
    ('exp_012', 0.0090, 0.0913),
    ('exp_024', 0.0087, 0.0893),
    ('exp_026', 0.0085, 0.0887),
    ('exp_030', 0.0083, 0.0877),
    ('exp_035', 0.0098, 0.0970),
]

from scipy import stats

cv_scores = np.array([s[1] for s in submissions])
lb_scores = np.array([s[2] for s in submissions])

# Linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(cv_scores, lb_scores)

print(f'Linear fit: LB = {slope:.4f} * CV + {intercept:.4f}')
print(f'R-squared = {r_value**2:.4f}')
print(f'Intercept = {intercept:.4f}')
print()

# Target analysis
target = 0.0347
print(f'Target LB: {target}')
print(f'Required CV to hit target: (target - intercept) / slope = ({target} - {intercept:.4f}) / {slope:.4f} = {(target - intercept) / slope:.6f}')
print()

# Predicted LB for exp_050
cv_050 = 0.008092
predicted_lb = slope * cv_050 + intercept
print(f'exp_050 CV: {cv_050}')
print(f'Predicted LB: {predicted_lb:.4f}')
print(f'Best LB so far: 0.0877 (exp_030)')

In [ ]:
# Key insight: The intercept problem
print('='*60)
print('KEY INSIGHT: THE INTERCEPT PROBLEM')
print('='*60)
print()
print(f'Intercept: {intercept:.4f}')
print(f'Target: {target}')
print()
if intercept > target:
    print('CRITICAL: The intercept is HIGHER than the target!')
    print('This means even with CV=0 (perfect training), LB would be ~{:.4f}'.format(intercept))
    print('The target is BELOW the intercept - unreachable with current approach.')
    print()
    print('Required CV to hit target: {:.6f} (NEGATIVE - impossible)'.format((target - intercept) / slope))
else:
    print('The intercept is lower than the target.')
    print('Required CV to hit target: {:.6f}'.format((target - intercept) / slope))